In [1]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable,RunnableParallel,RunnableBranch,RunnableLambda
from pydantic import BaseModel,Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser


In [2]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

load_dotenv()

llm1 = ChatGroq(
    model="moonshotai/kimi-k2-instruct-0905",  # ✅ spelling fixed
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.3,
)

response = llm1.invoke("Hi")
print(response.content)


Hi there! How can I help you today?


In [7]:
parser=StrOutputParser()

In [8]:
class feedback(BaseModel):
    sentiment:Literal['positive','negative']=Field(description="give the sentiment of the feedback")
    

In [9]:
parser2=PydanticOutputParser(pydantic_object=feedback)

In [10]:
p1=PromptTemplate(
    template="classify the senitment of the following tect into positive or negative \n {feedback} \n {format_instruction}",
    input_variables=['feesback'],
    partial_variables={'format_instruction':parser2.get_format_instructions}
)

In [11]:
classifier_chain=p1 | llm1 | parser2

In [12]:
p2=PromptTemplate(
    template="write an appropriate responcse to this positive feedback \n {feedback}",
    input_variables=['feesback']
)


In [13]:
p3=PromptTemplate(
    template="write an appropriate responcse to this negative feedback \n {feedback}",
    input_variables=['feesback']
)



In [14]:
branch_chain=RunnableBranch(
    (lambda x:x.sentiment == "positive", p2 | llm1 | parser ),
    (lambda x:x.sentiment == "negative", p3 | llm1 | parser),
    RunnableLambda(lambda X:"could not find sentiment")
)

In [15]:
chian= classifier_chain | branch_chain

In [16]:

print(chian.invoke({"feedback":"this is beautiful phone"}))

Sure! Here's a general response you can use for positive feedback:

---

Thank you so much for the kind words! I'm really glad to hear you had a positive experience. Your feedback means a lot, and I appreciate the support. If there's anything else I can help with, just let me know!

---

If you’d like to tailor it to a specific situation (like a product, service, or interaction), feel free to share more details!
